In [38]:
import pandas as pd
import numpy as np
from scipy import sparse
from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from tqdm.notebook import tqdm
import pickle

In [5]:
train_joke_df = pd.read_csv(r'..\data\recsys-in-practice\train_joke_df.csv')

joke_df = pd.read_csv(r'..\part1\doc2vec\joke_doc2vec_vectors.csv')

In [6]:
train_joke_df["UID"] = train_joke_df["UID"].astype(int)
train_joke_df["JID"] = train_joke_df["JID"].astype(int)

In [7]:
train_joke_df = train_joke_df.merge(joke_df, left_on='JID', right_index=True)
train_joke_df

,UID,JID,Rating,joke_feature_1,joke_feature_2,joke_feature_3,joke_feature_4,joke_feature_5,joke_feature_6,joke_feature_7,...,joke_feature_91,joke_feature_92,joke_feature_93,joke_feature_94,joke_feature_95,joke_feature_96,joke_feature_97,joke_feature_98,joke_feature_99,joke_feature_100
0,18029,6,-1.26,-0.036663,-0.067076,0.023498,-0.033260,0.055063,-0.029962,0.007167,...,-0.057065,-0.026336,-0.019614,-0.023196,0.053547,0.026594,0.018127,-0.023290,-0.042454,-0.008761
17,5621,6,3.16,-0.036663,-0.067076,0.023498,-0.033260,0.055063,-0.029962,0.007167,...,-0.057065,-0.026336,-0.019614,-0.023196,0.053547,0.026594,0.018127,-0.023290,-0.042454,-0.008761
101,17251,6,-3.74,-0.036663,-0.067076,0.023498,-0.033260,0.055063,-0.029962,0.007167,...,-0.057065,-0.026336,-0.019614,-0.023196,0.053547,0.026594,0.018127,-0.023290,-0.042454,-0.008761
142,2786,6,6.36,-0.036663,-0.067076,0.023498,-0.033260,0.055063,-0.029962,0.007167,...,-0.057065,-0.026336,-0.019614,-0.023196,0.053547,0.026594,0.018127,-0.023290,-0.042454,-0.008761
273,14079,6,2.09,-0.036663,-0.067076,0.023498,-0.033260,0.055063,-0.029962,0.007167,...,-0.057065,-0.026336,-0.019614,-0.023196,0.053547,0.026594,0.018127,-0.023290,-0.042454,-0.008761
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1447099,22356,74,1.89,0.004980,0.003973,-0.010549,-0.004713,0.029614,0.002287,-0.001566,...,-0.014091,-0.017337,-0.000019,-0.010038,0.009264,0.035359,0.022170,-0.013675,0.009155,-0.004111
1447290,4234,74,7.04,0.004980,0.003973,-0.010549,-0.004713,0.029614,0.002287,-0.001566,...,-0.014091,-0.017337,-0.000019,-0.010038,0.009264,0.035359,0.022170,-0.013675,0.009155,-0.004111
1447538,16115,74,0.34,0.004980,0.003973,-0.010549,-0.004713,0.029614,0.002287,-0.001566,...,-0.014091,-0.017337,-0.000019,-0.010038,0.009264,0.035359,0.022170,-0.013675,0.009155,-0.004111
1447727,11579,74,6.17,0.004980,0.003973,-0.010549,-0.004713,0.029614,0.002287,-0.001566,...,-0.014091,-0.017337,-0.000019,-0.010038,0.009264,0.035359,0.022170,-0.013675,0.009155,-0.004111


In [8]:
train_df, valid_df = train_test_split(train_joke_df, test_size=0.5, random_state=42)
train_df["UID"] = train_df["UID"].astype(int)
train_df["JID"] = train_df["JID"].astype(int)
valid_df["UID"] = valid_df["UID"].astype(int)
valid_df["JID"] = valid_df["JID"].astype(int)

In [9]:
# сделаем сортировку и перепишем index
train_df = train_df.sort_values(by=['UID', 'JID'])
train_df = train_df.reset_index(drop=True)

valid_df = valid_df.sort_values(by=['UID', 'JID'])
valid_df = valid_df.reset_index(drop=True)

In [10]:
from catboost import CatBoostRanker, Pool, MetricVisualizer, CatBoostRegressor
from copy import deepcopy

In [11]:
cat_features = ['UID', 'JID']

In [12]:
train_pool = Pool(train_df.drop(columns='Rating'), label=train_df['Rating'], group_id=train_df['UID'],cat_features=cat_features)
valid_pool = Pool(valid_df.drop(columns='Rating'), label=valid_df['Rating'], group_id=valid_df['UID'],cat_features=cat_features)

In [13]:
default_parameters = {
    'iterations': 2000,
    'custom_metric': 'RMSE',
    'random_seed': 0,
    'train_dir':'RMSE',
    'objective':'RMSE',
    'loss_function':'RMSE',
    'eval_metric':'RMSE'
}


In [14]:
model = CatBoostRegressor(**default_parameters)
model.fit(train_pool, eval_set=valid_pool, plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Learning rate set to 0.093732
0:	learn: 5.1380028	test: 5.1243726	best: 5.1243726 (0)	total: 518ms	remaining: 17m 14s
1:	learn: 5.0564060	test: 5.0325277	best: 5.0325277 (1)	total: 688ms	remaining: 11m 27s
2:	learn: 4.9885391	test: 4.9537839	best: 4.9537839 (2)	total: 946ms	remaining: 10m 29s
3:	learn: 4.9296650	test: 4.8862962	best: 4.8862962 (3)	total: 1.11s	remaining: 9m 12s
4:	learn: 4.8803754	test: 4.8299843	best: 4.8299843 (4)	total: 1.41s	remaining: 9m 21s
5:	learn: 4.8378240	test: 4.7800292	best: 4.7800292 (5)	total: 1.57s	remaining: 8m 42s
6:	learn: 4.8025761	test: 4.7383124	best: 4.7383124 (6)	total: 1.69s	remaining: 8m 2s
7:	learn: 4.7727960	test: 4.7027357	best: 4.7027357 (7)	total: 1.85s	remaining: 7m 39s
8:	learn: 4.7464854	test: 4.6718082	best: 4.6718082 (8)	total: 2.01s	remaining: 7m 25s
9:	learn: 4.7249214	test: 4.6460182	best: 4.6460182 (9)	total: 2.16s	remaining: 7m 10s
10:	learn: 4.7059087	test: 4.6226851	best: 4.6226851 (10)	total: 2.33s	remaining: 7m
11:	learn: 4.

93:	learn: 4.5792831	test: 4.4546929	best: 4.4546929 (93)	total: 24s	remaining: 8m 6s
94:	learn: 4.5791196	test: 4.4545193	best: 4.4545193 (94)	total: 24.4s	remaining: 8m 9s
95:	learn: 4.5790884	test: 4.4544885	best: 4.4544885 (95)	total: 24.6s	remaining: 8m 8s
96:	learn: 4.5789407	test: 4.4542654	best: 4.4542654 (96)	total: 25s	remaining: 8m 11s
97:	learn: 4.5788246	test: 4.4541489	best: 4.4541489 (97)	total: 25.5s	remaining: 8m 14s
98:	learn: 4.5786842	test: 4.4539848	best: 4.4539848 (98)	total: 25.9s	remaining: 8m 16s
99:	learn: 4.5785954	test: 4.4539092	best: 4.4539092 (99)	total: 26.4s	remaining: 8m 21s
100:	learn: 4.5785293	test: 4.4538519	best: 4.4538519 (100)	total: 26.7s	remaining: 8m 21s
101:	learn: 4.5784346	test: 4.4537483	best: 4.4537483 (101)	total: 27.1s	remaining: 8m 24s
102:	learn: 4.5783110	test: 4.4536601	best: 4.4536601 (102)	total: 27.6s	remaining: 8m 28s
103:	learn: 4.5782119	test: 4.4535561	best: 4.4535561 (103)	total: 27.8s	remaining: 8m 27s
104:	learn: 4.578116

185:	learn: 4.5718755	test: 4.4479897	best: 4.4479897 (185)	total: 55.6s	remaining: 9m 2s
186:	learn: 4.5717826	test: 4.4479283	best: 4.4479283 (186)	total: 56.1s	remaining: 9m 4s
187:	learn: 4.5716975	test: 4.4478425	best: 4.4478425 (187)	total: 56.3s	remaining: 9m 3s
188:	learn: 4.5716814	test: 4.4478205	best: 4.4478205 (188)	total: 56.7s	remaining: 9m 2s
189:	learn: 4.5714498	test: 4.4475620	best: 4.4475620 (189)	total: 57.1s	remaining: 9m 4s
190:	learn: 4.5714068	test: 4.4475386	best: 4.4475386 (190)	total: 57.6s	remaining: 9m 5s
191:	learn: 4.5713684	test: 4.4475095	best: 4.4475095 (191)	total: 57.9s	remaining: 9m 5s
192:	learn: 4.5713324	test: 4.4474933	best: 4.4474933 (192)	total: 58.1s	remaining: 9m 3s
193:	learn: 4.5712692	test: 4.4474821	best: 4.4474821 (193)	total: 58.4s	remaining: 9m 3s
194:	learn: 4.5711827	test: 4.4473885	best: 4.4473885 (194)	total: 58.9s	remaining: 9m 5s
195:	learn: 4.5710953	test: 4.4473214	best: 4.4473214 (195)	total: 59.2s	remaining: 9m 5s
196:	learn

276:	learn: 4.5664205	test: 4.4440012	best: 4.4440012 (276)	total: 1m 26s	remaining: 8m 56s
277:	learn: 4.5663812	test: 4.4439303	best: 4.4439303 (277)	total: 1m 26s	remaining: 8m 57s
278:	learn: 4.5663469	test: 4.4439102	best: 4.4439102 (278)	total: 1m 26s	remaining: 8m 55s
279:	learn: 4.5663143	test: 4.4439111	best: 4.4439102 (278)	total: 1m 27s	remaining: 8m 56s
280:	learn: 4.5662915	test: 4.4438969	best: 4.4438969 (280)	total: 1m 27s	remaining: 8m 56s
281:	learn: 4.5662650	test: 4.4438792	best: 4.4438792 (281)	total: 1m 27s	remaining: 8m 55s
282:	learn: 4.5662523	test: 4.4438607	best: 4.4438607 (282)	total: 1m 28s	remaining: 8m 55s
283:	learn: 4.5661995	test: 4.4438130	best: 4.4438130 (283)	total: 1m 28s	remaining: 8m 54s
284:	learn: 4.5661215	test: 4.4438123	best: 4.4438123 (284)	total: 1m 28s	remaining: 8m 53s
285:	learn: 4.5660527	test: 4.4437825	best: 4.4437825 (285)	total: 1m 29s	remaining: 8m 55s
286:	learn: 4.5660152	test: 4.4437721	best: 4.4437721 (286)	total: 1m 29s	remain

...

1612:	learn: 4.5207933	test: 4.4209356	best: 4.4209356 (1612)	total: 9m 46s	remaining: 2m 20s
1613:	learn: 4.5207820	test: 4.4209347	best: 4.4209347 (1613)	total: 9m 47s	remaining: 2m 20s
1614:	learn: 4.5207734	test: 4.4209384	best: 4.4209347 (1613)	total: 9m 47s	remaining: 2m 20s
1615:	learn: 4.5207585	test: 4.4209296	best: 4.4209296 (1615)	total: 9m 47s	remaining: 2m 19s
1616:	learn: 4.5207388	test: 4.4209217	best: 4.4209217 (1616)	total: 9m 48s	remaining: 2m 19s
1617:	learn: 4.5207333	test: 4.4209237	best: 4.4209217 (1616)	total: 9m 48s	remaining: 2m 18s
1618:	learn: 4.5206881	test: 4.4208705	best: 4.4208705 (1618)	total: 9m 49s	remaining: 2m 18s
1619:	learn: 4.5206656	test: 4.4208670	best: 4.4208670 (1619)	total: 9m 49s	remaining: 2m 18s
1620:	learn: 4.5206259	test: 4.4208625	best: 4.4208625 (1620)	total: 9m 49s	remaining: 2m 17s
1621:	learn: 4.5206057	test: 4.4208639	best: 4.4208625 (1620)	total: 9m 50s	remaining: 2m 17s
1622:	learn: 4.5205775	test: 4.4208659	best: 4.4208625 (1620

1700:	learn: 4.5183861	test: 4.4201273	best: 4.4201273 (1700)	total: 10m 19s	remaining: 1m 48s
1701:	learn: 4.5183215	test: 4.4200425	best: 4.4200425 (1701)	total: 10m 20s	remaining: 1m 48s
1702:	learn: 4.5182950	test: 4.4200342	best: 4.4200342 (1702)	total: 10m 20s	remaining: 1m 48s
1703:	learn: 4.5182636	test: 4.4200273	best: 4.4200273 (1703)	total: 10m 20s	remaining: 1m 47s
1704:	learn: 4.5182447	test: 4.4200167	best: 4.4200167 (1704)	total: 10m 21s	remaining: 1m 47s
1705:	learn: 4.5182166	test: 4.4200152	best: 4.4200152 (1705)	total: 10m 21s	remaining: 1m 47s
1706:	learn: 4.5182045	test: 4.4200180	best: 4.4200152 (1705)	total: 10m 21s	remaining: 1m 46s
1707:	learn: 4.5181801	test: 4.4200125	best: 4.4200125 (1707)	total: 10m 22s	remaining: 1m 46s
1708:	learn: 4.5181624	test: 4.4200110	best: 4.4200110 (1708)	total: 10m 22s	remaining: 1m 45s
1709:	learn: 4.5181397	test: 4.4199950	best: 4.4199950 (1709)	total: 10m 22s	remaining: 1m 45s
1710:	learn: 4.5181231	test: 4.4199978	best: 4.419

1788:	learn: 4.5159323	test: 4.4190933	best: 4.4190933 (1788)	total: 10m 54s	remaining: 1m 17s
1789:	learn: 4.5159063	test: 4.4191006	best: 4.4190933 (1788)	total: 10m 54s	remaining: 1m 16s
1790:	learn: 4.5158809	test: 4.4190952	best: 4.4190933 (1788)	total: 10m 55s	remaining: 1m 16s
1791:	learn: 4.5158667	test: 4.4190877	best: 4.4190877 (1791)	total: 10m 55s	remaining: 1m 16s
1792:	learn: 4.5158422	test: 4.4190743	best: 4.4190743 (1792)	total: 10m 56s	remaining: 1m 15s
1793:	learn: 4.5158264	test: 4.4190732	best: 4.4190732 (1793)	total: 10m 56s	remaining: 1m 15s
1794:	learn: 4.5157941	test: 4.4190585	best: 4.4190585 (1794)	total: 10m 56s	remaining: 1m 15s
1795:	learn: 4.5157418	test: 4.4190149	best: 4.4190149 (1795)	total: 10m 57s	remaining: 1m 14s
1796:	learn: 4.5157078	test: 4.4190067	best: 4.4190067 (1796)	total: 10m 57s	remaining: 1m 14s
1797:	learn: 4.5156999	test: 4.4190110	best: 4.4190067 (1796)	total: 10m 58s	remaining: 1m 13s
1798:	learn: 4.5156636	test: 4.4190032	best: 4.419

1876:	learn: 4.5137933	test: 4.4183745	best: 4.4183745 (1876)	total: 11m 27s	remaining: 45.1s
1877:	learn: 4.5137574	test: 4.4183598	best: 4.4183598 (1877)	total: 11m 27s	remaining: 44.7s
1878:	learn: 4.5137435	test: 4.4183635	best: 4.4183598 (1877)	total: 11m 28s	remaining: 44.3s
1879:	learn: 4.5136239	test: 4.4182123	best: 4.4182123 (1879)	total: 11m 28s	remaining: 44s
1880:	learn: 4.5135829	test: 4.4181743	best: 4.4181743 (1880)	total: 11m 29s	remaining: 43.6s
1881:	learn: 4.5135345	test: 4.4181169	best: 4.4181169 (1881)	total: 11m 29s	remaining: 43.3s
1882:	learn: 4.5134528	test: 4.4180476	best: 4.4180476 (1882)	total: 11m 30s	remaining: 42.9s
1883:	learn: 4.5134366	test: 4.4180370	best: 4.4180370 (1883)	total: 11m 30s	remaining: 42.5s
1884:	learn: 4.5133951	test: 4.4180100	best: 4.4180100 (1884)	total: 11m 31s	remaining: 42.2s
1885:	learn: 4.5133491	test: 4.4180053	best: 4.4180053 (1885)	total: 11m 31s	remaining: 41.8s
1886:	learn: 4.5133353	test: 4.4179967	best: 4.4179967 (1886)	

1964:	learn: 4.5116271	test: 4.4174691	best: 4.4174604 (1957)	total: 11m 59s	remaining: 12.8s
1965:	learn: 4.5116041	test: 4.4174603	best: 4.4174603 (1965)	total: 11m 59s	remaining: 12.5s
1966:	learn: 4.5116002	test: 4.4174602	best: 4.4174602 (1966)	total: 12m	remaining: 12.1s
1967:	learn: 4.5115868	test: 4.4174618	best: 4.4174602 (1966)	total: 12m	remaining: 11.7s
1968:	learn: 4.5115557	test: 4.4174552	best: 4.4174552 (1968)	total: 12m	remaining: 11.3s
1969:	learn: 4.5115432	test: 4.4174536	best: 4.4174536 (1969)	total: 12m 1s	remaining: 11s
1970:	learn: 4.5115255	test: 4.4174556	best: 4.4174536 (1969)	total: 12m 1s	remaining: 10.6s
1971:	learn: 4.5114912	test: 4.4174443	best: 4.4174443 (1971)	total: 12m 2s	remaining: 10.3s
1972:	learn: 4.5114617	test: 4.4174466	best: 4.4174443 (1971)	total: 12m 2s	remaining: 9.89s
1973:	learn: 4.5114248	test: 4.4174096	best: 4.4174096 (1973)	total: 12m 3s	remaining: 9.52s
1974:	learn: 4.5114160	test: 4.4174135	best: 4.4174096 (1973)	total: 12m 3s	rem

In [15]:
predict = model.predict(valid_pool)
print(mean_squared_error(valid_df['Rating'].values, predict, squared=False))

predict = model.predict(train_pool)
print(mean_squared_error(train_df['Rating'].values, predict, squared=False))


4.41730049584738
4.217502444175489


In [16]:
model_name = "catboost_doc2vec"

In [19]:
model.save_model(model_name)
from_file = CatBoostRegressor()

from_file.load_model(model_name)


In [20]:
predict = from_file.predict(valid_pool)
print(mean_squared_error(valid_df['Rating'].values, predict, squared=False))

predict = from_file.predict(train_pool)
print(mean_squared_error(train_df['Rating'].values, predict, squared=False))


4.41730049584738
4.217502444175489


In [23]:
n_users = np.unique(train_joke_df['UID']).size
n_items = np.unique(train_joke_df['JID']).size

test_set_all = np.zeros((n_users * n_items, 2))
for u in tqdm(range(n_users)):
    for j in range(n_items):
        test_set_all[(n_items * u) + j, 0] = u
        test_set_all[(n_items * u) + j, 1] = j

df_test = pd.DataFrame(test_set_all, columns=['UID', 'JID'])
df_test

  0%|          | 0/24983 [00:00<?, ?it/s]

,UID,JID
0,0.0,0.0
1,0.0,1.0
2,0.0,2.0
3,0.0,3.0
4,0.0,4.0
...,...,...
2473312,24982.0,94.0
2473313,24982.0,95.0
2473314,24982.0,96.0
2473315,24982.0,97.0


In [24]:

df_test['UID'] = df_test['UID'].astype(int)
df_test['JID'] = df_test['JID'].astype(int)

In [25]:

df_test = df_test.merge(joke_df, how='left', left_on='JID', right_index=True)
display(df_test)
df_test = df_test.sort_values(by=['UID', 'JID'])

,UID,JID,joke_feature_1,joke_feature_2,joke_feature_3,joke_feature_4,joke_feature_5,joke_feature_6,joke_feature_7,joke_feature_8,...,joke_feature_91,joke_feature_92,joke_feature_93,joke_feature_94,joke_feature_95,joke_feature_96,joke_feature_97,joke_feature_98,joke_feature_99,joke_feature_100
0,0,0,0.019126,0.012131,-0.027477,0.019825,-0.007704,0.020013,-0.022230,-0.015224,...,0.013015,0.027021,-0.012433,0.006457,-0.014875,-0.015005,-0.018308,-0.001017,0.048864,0.007200
1,0,1,-0.011499,0.002285,0.003630,0.004051,0.003290,-0.000463,0.006562,-0.001884,...,-0.009672,-0.005080,-0.002575,-0.005094,0.006556,0.001787,0.013340,0.011139,-0.017809,-0.000959
2,0,2,-0.016668,-0.018199,0.012311,-0.010475,0.025088,0.000381,0.004331,0.023751,...,-0.019020,-0.012687,-0.009148,-0.009461,0.019125,0.026595,0.013386,-0.013205,-0.019013,-0.015191
3,0,3,-0.032417,0.002634,0.027978,-0.019963,0.023566,0.007097,0.021878,0.044580,...,-0.016706,-0.034643,0.006712,-0.007995,0.028974,0.030824,0.034675,0.001362,-0.046984,-0.017229
4,0,4,-0.014843,-0.046528,0.002575,-0.029940,0.030619,-0.011654,0.012663,0.048356,...,-0.025209,-0.016499,-0.004959,-0.026612,0.021617,0.048349,0.002544,-0.042855,-0.021552,-0.023502
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2473312,24982,94,-0.000154,-0.006946,-0.010851,0.011448,0.004269,-0.007663,0.003276,-0.000208,...,-0.003951,-0.009226,-0.001803,-0.025048,0.003969,0.020448,0.011776,-0.013034,-0.003493,-0.020192
2473313,24982,95,-0.004857,0.012683,0.009205,-0.008275,-0.003818,0.006676,-0.003004,0.014902,...,0.011361,-0.006212,0.008240,-0.001582,0.010192,-0.002656,0.002402,0.000192,-0.013342,-0.000508
2473314,24982,96,0.034768,0.040293,-0.008734,0.023405,-0.038429,0.024317,-0.039087,-0.039802,...,0.036261,0.070944,-0.018116,0.070075,-0.064077,-0.088195,-0.061626,0.055512,0.110628,0.040841
2473315,24982,97,0.017260,0.013419,-0.001536,-0.003390,-0.020430,-0.008701,0.003307,-0.016211,...,0.025377,-0.005499,0.053408,0.017966,0.011246,-0.036009,0.007395,0.013810,-0.027661,0.000457


In [26]:

test_pool = Pool(df_test, group_id=df_test['UID'], cat_features=cat_features)



In [27]:
predictions = model.predict(test_pool)


In [28]:
df_test['Rating_pred'] = predictions
df_test

,UID,JID,joke_feature_1,joke_feature_2,joke_feature_3,joke_feature_4,joke_feature_5,joke_feature_6,joke_feature_7,joke_feature_8,...,joke_feature_92,joke_feature_93,joke_feature_94,joke_feature_95,joke_feature_96,joke_feature_97,joke_feature_98,joke_feature_99,joke_feature_100,Rating_pred
0,0,0,0.019126,0.012131,-0.027477,0.019825,-0.007704,0.020013,-0.022230,-0.015224,...,0.027021,-0.012433,0.006457,-0.014875,-0.015005,-0.018308,-0.001017,0.048864,0.007200,3.297993
1,0,1,-0.011499,0.002285,0.003630,0.004051,0.003290,-0.000463,0.006562,-0.001884,...,-0.005080,-0.002575,-0.005094,0.006556,0.001787,0.013340,0.011139,-0.017809,-0.000959,3.497294
2,0,2,-0.016668,-0.018199,0.012311,-0.010475,0.025088,0.000381,0.004331,0.023751,...,-0.012687,-0.009148,-0.009461,0.019125,0.026595,0.013386,-0.013205,-0.019013,-0.015191,5.748212
3,0,3,-0.032417,0.002634,0.027978,-0.019963,0.023566,0.007097,0.021878,0.044580,...,-0.034643,0.006712,-0.007995,0.028974,0.030824,0.034675,0.001362,-0.046984,-0.017229,3.986693
4,0,4,-0.014843,-0.046528,0.002575,-0.029940,0.030619,-0.011654,0.012663,0.048356,...,-0.016499,-0.004959,-0.026612,0.021617,0.048349,0.002544,-0.042855,-0.021552,-0.023502,3.870478
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2473312,24982,94,-0.000154,-0.006946,-0.010851,0.011448,0.004269,-0.007663,0.003276,-0.000208,...,-0.009226,-0.001803,-0.025048,0.003969,0.020448,0.011776,-0.013034,-0.003493,-0.020192,-0.008525
2473313,24982,95,-0.004857,0.012683,0.009205,-0.008275,-0.003818,0.006676,-0.003004,0.014902,...,-0.006212,0.008240,-0.001582,0.010192,-0.002656,0.002402,0.000192,-0.013342,-0.000508,0.413416
2473314,24982,96,0.034768,0.040293,-0.008734,0.023405,-0.038429,0.024317,-0.039087,-0.039802,...,0.070944,-0.018116,0.070075,-0.064077,-0.088195,-0.061626,0.055512,0.110628,0.040841,0.496009
2473315,24982,97,0.017260,0.013419,-0.001536,-0.003390,-0.020430,-0.008701,0.003307,-0.016211,...,-0.005499,0.053408,0.017966,0.011246,-0.036009,0.007395,0.013810,-0.027661,0.000457,1.273655


In [29]:
mrg = df_test.merge(train_df, on=['UID', 'JID'], how="left", indicator=True)
mrg

,UID,JID,joke_feature_1_x,joke_feature_2_x,joke_feature_3_x,joke_feature_4_x,joke_feature_5_x,joke_feature_6_x,joke_feature_7_x,joke_feature_8_x,...,joke_feature_92_y,joke_feature_93_y,joke_feature_94_y,joke_feature_95_y,joke_feature_96_y,joke_feature_97_y,joke_feature_98_y,joke_feature_99_y,joke_feature_100_y,_merge
0,0,0,0.019126,0.012131,-0.027477,0.019825,-0.007704,0.020013,-0.022230,-0.015224,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
1,0,1,-0.011499,0.002285,0.003630,0.004051,0.003290,-0.000463,0.006562,-0.001884,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
2,0,2,-0.016668,-0.018199,0.012311,-0.010475,0.025088,0.000381,0.004331,0.023751,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
3,0,3,-0.032417,0.002634,0.027978,-0.019963,0.023566,0.007097,0.021878,0.044580,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
4,0,4,-0.014843,-0.046528,0.002575,-0.029940,0.030619,-0.011654,0.012663,0.048356,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2473312,24982,94,-0.000154,-0.006946,-0.010851,0.011448,0.004269,-0.007663,0.003276,-0.000208,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
2473313,24982,95,-0.004857,0.012683,0.009205,-0.008275,-0.003818,0.006676,-0.003004,0.014902,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
2473314,24982,96,0.034768,0.040293,-0.008734,0.023405,-0.038429,0.024317,-0.039087,-0.039802,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
2473315,24982,97,0.017260,0.013419,-0.001536,-0.003390,-0.020430,-0.008701,0.003307,-0.016211,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only


In [30]:
mrg = mrg[mrg['_merge'] == 'left_only']
mrg

,UID,JID,joke_feature_1_x,joke_feature_2_x,joke_feature_3_x,joke_feature_4_x,joke_feature_5_x,joke_feature_6_x,joke_feature_7_x,joke_feature_8_x,...,joke_feature_92_y,joke_feature_93_y,joke_feature_94_y,joke_feature_95_y,joke_feature_96_y,joke_feature_97_y,joke_feature_98_y,joke_feature_99_y,joke_feature_100_y,_merge
0,0,0,0.019126,0.012131,-0.027477,0.019825,-0.007704,0.020013,-0.022230,-0.015224,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
1,0,1,-0.011499,0.002285,0.003630,0.004051,0.003290,-0.000463,0.006562,-0.001884,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
2,0,2,-0.016668,-0.018199,0.012311,-0.010475,0.025088,0.000381,0.004331,0.023751,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
3,0,3,-0.032417,0.002634,0.027978,-0.019963,0.023566,0.007097,0.021878,0.044580,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
4,0,4,-0.014843,-0.046528,0.002575,-0.029940,0.030619,-0.011654,0.012663,0.048356,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2473312,24982,94,-0.000154,-0.006946,-0.010851,0.011448,0.004269,-0.007663,0.003276,-0.000208,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
2473313,24982,95,-0.004857,0.012683,0.009205,-0.008275,-0.003818,0.006676,-0.003004,0.014902,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
2473314,24982,96,0.034768,0.040293,-0.008734,0.023405,-0.038429,0.024317,-0.039087,-0.039802,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
2473315,24982,97,0.017260,0.013419,-0.001536,-0.003390,-0.020430,-0.008701,0.003307,-0.016211,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only


In [31]:
def get_n_recommendations_for_user(df, user_id, n, sort_by):
    recommended_items = df[df['UID'] == user_id]
    recommended_items = recommended_items.sort_values(sort_by, ascending=False)  
    return recommended_items.iloc[:n]

In [32]:
n_recommendations = 10

In [33]:
frames = []
for user in tqdm(range(n_users)):
    frames.append(get_n_recommendations_for_user(mrg, user, n_recommendations, 'Rating_pred'))
    
df_rec = pd.concat(frames).reset_index()
df_rec = df_rec.drop(columns=['index'])
df_rec

  0%|          | 0/24983 [00:00<?, ?it/s]

,UID,JID,joke_feature_1_x,joke_feature_2_x,joke_feature_3_x,joke_feature_4_x,joke_feature_5_x,joke_feature_6_x,joke_feature_7_x,joke_feature_8_x,...,joke_feature_92_y,joke_feature_93_y,joke_feature_94_y,joke_feature_95_y,joke_feature_96_y,joke_feature_97_y,joke_feature_98_y,joke_feature_99_y,joke_feature_100_y,_merge
0,0,35,-0.033016,-0.022457,0.018897,-0.027672,0.037534,-0.006529,0.009649,0.038965,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
1,0,38,0.004627,0.005074,-0.004401,0.010018,-0.008095,0.004768,-0.017488,-0.008671,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
2,0,89,-0.061068,-0.044929,0.062493,-0.046146,0.059271,-0.023593,0.042571,0.111263,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
3,0,6,-0.036663,-0.067076,0.023498,-0.033260,0.055063,-0.029962,0.007167,0.014843,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
4,0,47,-0.010867,-0.000013,0.012010,-0.002431,0.009287,-0.000874,0.006839,0.026251,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249825,24982,53,0.015199,0.006955,-0.026504,0.018141,-0.021094,0.002491,0.001258,-0.031133,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
249826,24982,69,0.002121,-0.029449,-0.013974,-0.010344,0.027003,-0.011740,0.006695,0.002539,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
249827,24982,62,-0.013547,0.011272,0.014504,0.005440,-0.011418,-0.024910,-0.009044,-0.016531,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
249828,24982,42,0.041332,0.027141,-0.089489,-0.009382,-0.049431,-0.010458,0.083823,-0.065743,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only


In [34]:
df_rec.merge(train_df, on=['UID','JID'])

,UID,JID,joke_feature_1_x,joke_feature_2_x,joke_feature_3_x,joke_feature_4_x,joke_feature_5_x,joke_feature_6_x,joke_feature_7_x,joke_feature_8_x,...,joke_feature_91,joke_feature_92,joke_feature_93,joke_feature_94,joke_feature_95,joke_feature_96,joke_feature_97,joke_feature_98,joke_feature_99,joke_feature_100


In [35]:
df_rec.merge(valid_df, on=['UID','JID'])

,UID,JID,joke_feature_1_x,joke_feature_2_x,joke_feature_3_x,joke_feature_4_x,joke_feature_5_x,joke_feature_6_x,joke_feature_7_x,joke_feature_8_x,...,joke_feature_91,joke_feature_92,joke_feature_93,joke_feature_94,joke_feature_95,joke_feature_96,joke_feature_97,joke_feature_98,joke_feature_99,joke_feature_100
0,1,36,-0.001037,0.006190,-0.008002,0.009853,-0.003585,0.004941,0.003574,-0.010647,...,0.009863,0.006131,0.004225,-0.001143,-0.008263,-0.013172,-0.005562,0.007160,-0.000393,0.000634
1,1,32,0.012997,-0.019345,0.001703,0.007861,0.000268,-0.022598,-0.003993,-0.019360,...,-0.019923,-0.011656,0.026532,-0.005845,0.005256,-0.000017,-0.005756,0.003037,-0.005521,0.007523
2,1,53,0.015199,0.006955,-0.026504,0.018141,-0.021094,0.002491,0.001258,-0.031133,...,0.011483,0.014364,0.006693,-0.004073,-0.016496,-0.017518,-0.012153,0.005795,0.013575,0.006137
3,1,62,-0.013547,0.011272,0.014504,0.005440,-0.011418,-0.024910,-0.009044,-0.016531,...,0.013551,0.003050,-0.011050,0.025457,0.036402,-0.043246,-0.004150,0.011129,0.018653,0.030358
4,1,68,0.024309,0.001024,-0.026021,0.033786,-0.014048,-0.003762,-0.008327,-0.045336,...,-0.003169,0.005297,0.019142,-0.008203,0.000500,-0.023445,-0.012393,0.009742,-0.010909,0.009591
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143829,24982,36,-0.001037,0.006190,-0.008002,0.009853,-0.003585,0.004941,0.003574,-0.010647,...,0.009863,0.006131,0.004225,-0.001143,-0.008263,-0.013172,-0.005562,0.007160,-0.000393,0.000634
143830,24982,29,0.003673,0.001946,-0.014492,-0.002926,0.011435,-0.000562,0.013989,0.011275,...,0.010610,0.010710,-0.013466,-0.005950,-0.026658,0.012785,-0.008919,-0.040272,0.033426,0.008710
143831,24982,53,0.015199,0.006955,-0.026504,0.018141,-0.021094,0.002491,0.001258,-0.031133,...,0.011483,0.014364,0.006693,-0.004073,-0.016496,-0.017518,-0.012153,0.005795,0.013575,0.006137
143832,24982,69,0.002121,-0.029449,-0.013974,-0.010344,0.027003,-0.011740,0.006695,0.002539,...,-0.025404,-0.021444,0.002309,-0.033459,0.009489,0.027197,0.008491,-0.017330,-0.026968,-0.023833


In [36]:
catboost_predictions = list(df_rec.groupby('UID').agg({'JID':list})['JID'].values)

In [39]:
predictions_df_catboost_doc2vec = {'catboost_doc2vec':df_rec}
with open('predictions_df_catboost_doc2vec.pkl', 'wb') as f:
    pickle.dump(predictions_df_catboost_doc2vec, f)